# Filter each stationary frame

In [648]:
import itertools
import json
import os
import random
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from pathlib import Path
from utils.filter_eval import evaluate_filter
from collections import defaultdict

## Constants and such

In [650]:
# Set background map resolutions
azimuth_resolution = 0.1
height_resolution = 0.25
std_dev_cutoff = 1.5

In [651]:
DATA_DIR_ROOT = '../data'

In [652]:
# Set to true if you want to make a new distances table, otherwise, will load previous map
# Set everything to false if you just want to filter frames
make_new_distances_df = False
make_new_lookup_table = False
new_grid_search = False

## First, functions for making background map

In [654]:
# # Create mappings for azimuth and height using integers
# def create_mappings(azimuth_step=azimuth_resolution, height_step=height_resolution):
#     azimuth_range = np.arange(-180, 180 + azimuth_step, azimuth_step)
#     # print(azimuth_range)
#     height_range = np.arange(-30, 10 + height_step, height_step)
#     azimuth_map = {int(az * 10): idx for idx, az in enumerate(azimuth_range)}
#     height_map = {int(ht * 10): idx for idx, ht in enumerate(height_range)}
#     return azimuth_map, height_map

In [655]:
# test, _ = create_mappings()

In [656]:
# print(test)

In [657]:
# Create mappings for azimuth and height using integers
def create_mappings(azimuth_step=azimuth_resolution, height_step=height_resolution):
    # Create ranges and multiply by 10 to convert to integers
    azimuth_range = np.arange(-180, 180 + azimuth_step, azimuth_step) * 10
    print(azimuth_range)
    
    height_range = np.arange(-30, 10 + height_step, height_step) * 10
    
    # Convert to integers and create dictionaries to map azimuth/height to index
    azimuth_map = {int(az): idx for idx, az in enumerate(azimuth_range)}
    height_map = {int(ht): idx for idx, ht in enumerate(height_range)}
    return azimuth_map, height_map

In [658]:
# test, _ = create_mappings()

In [659]:
# Create the grid DataFrames
def create_grid_dataframes():
    azimuth_map, height_map = create_mappings()
    grid_shape = (len(height_map), len(azimuth_map))
    df_distances = pd.DataFrame({key: [[] for _ in range(len(height_map))] \
                                 for key in azimuth_map.keys()}, index=height_map.keys())
    # df_intensities = pd.DataFrame({key: [[] for _ in range(len(height_map))] \
                                   # for key in azimuth_map.keys()}, index=height_map.keys())
    return df_distances, azimuth_map, height_map

In [660]:
# test_df, _, _, _ = create_grid_dataframes()

In [661]:
# test_df

In [662]:
# # Process file into grid
# def process_files_to_grid(data_dir):
#     # Create empty grid
#     # df_distances, df_intensities, azimuth_map, height_map = create_grid_dataframes()
#     df_distances, df_intensities, azimuth_map, height_map = create_grid_dataframes()

#     lidar_dir = Path(data_dir, 'velodyne_points')

#     # Get a list of all the file paths
#     all_files = list(lidar_dir.iterdir())
    
#     # Randomly select 100 files from the list
#     files_to_process = random.sample(all_files, 1000)
    
#     # For each file in the random selection
#     for file_path in files_to_process:

#     # For each file in the directory
#     # for file_path in lidar_dir.iterdir():
#         print('.', end ='')
#         data = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
#         for x, y, z, intensity in data:
#             # Convert to azimuth, height, distance format
#             distance = np.sqrt(x**2 + y**2 + z**2)
#             azimuth = np.degrees(np.arctan2(y, x))
#             height = np.degrees(np.arctan2(z, np.sqrt(x**2 + y**2)))
#             # Convert and scale

#             azimuth_idx = (np.floor((azimuth + 180) / azimuth_resolution) * azimuth_resolution * 100 - 18000).astype(int)
#             height_idx = (np.floor((height + 30) / height_resolution) * height_resolution * 100 - 3000).astype(int)

            
#             # Update DataFrames directly using indices
#             if azimuth_idx in azimuth_map and height_idx in height_map:
#                 df_distances.at[height_idx, azimuth_idx].append(distance)
#     return df_distances

In [663]:
# Process file into grid
def process_files_to_grid(data_dir):
    # Create empty grid
    # df_distances, df_intensities, azimuth_map, height_map = create_grid_dataframes()
    df_z, azimuth_map, height_map = create_grid_dataframes()

    lidar_dir = Path(data_dir, 'velodyne_points')

    # Get a list of all the file paths
    all_files = list(lidar_dir.iterdir())
    
    # Randomly select 100 files from the list
    files_to_process = random.sample(all_files, 100)
    
    # For each file in the random selection
    for file_path in files_to_process:

    # For each file in the directory
    # for file_path in lidar_dir.iterdir():
        print('.', end ='')
        data = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
        for x, y, z, intensity in data:
            # Convert to azimuth, height, distance format
            distance = np.sqrt(x**2 + y**2 + z**2)
            azimuth = np.degrees(np.arctan2(y, x))
            height = np.degrees(np.arctan2(z, np.sqrt(x**2 + y**2)))
            # Convert and scale

            azimuth_idx = (np.floor((azimuth + 180) / azimuth_resolution) * azimuth_resolution * 10 - 1800).astype(int)
            height_idx = (np.floor((height + 30) / height_resolution) * height_resolution * 10 - 300).astype(int)

            
            # Update DataFrames directly using indices
            if azimuth_idx in azimuth_map and height_idx in height_map:
                df_z.at[height_idx, azimuth_idx].append(z)
    return df_z

In [664]:
def get_distances_dataframe(dir):
    # Get the distances of the background map from the lidar files
    # if make_new_map or not files_exist("df_distances.pkl", "df_intensities.pkl"):
    if make_new_distances_df:
        df_distances = process_files_to_grid(dir)

        return df_distances
    if not make_new_lookup_table:
        print('\nSkip loading dataframe')
        return pd.DataFrame()
    else:
        # Load the DataFrames
        print('\nLoading dataframe')
        df_distances = pd.read_pickle("df_heights.pkl")
        return df_distances
    

In [665]:
def files_exist(*files):
    return all(os.path.exists(file) for file in files)

In [666]:
def get_cutoff_base(cutoff_base_param, distances):
    if cutoff_base_param == 'average':
        return np.mean(distances)
    elif cutoff_base_param == 'first_quartile':
        return np.percentile(distances, 25)
    elif cutoff_base_param == 'median':
        return np.median(distances)
    elif cutoff_base_param == 'third_quartile':
        return np.percentile(distances, 75)
    elif cutoff_base_param == 'maximum':
        return np.max(distances)
    else:
        print(cutoff_base_param + ' is not a valid cutoff_base value')
    return 0

In [667]:
def get_adjustment_direction_param(adjustment_direction_param, distances):
    if adjustment_direction_param == 'add':
        return 1
    elif adjustment_direction_param == 'subtract':
        return -1
    else:
        print(adjustment_direction_param + ' is not a valid adjustment_direction_param value')
    return 0 

In [668]:
def get_adjustment_parameter_param(adjustment_param, distances):
    if adjustment_param == 'constant':
        return 1
    elif adjustment_param == 'standard_deviation':
        return np.std(distances)
    else:
        print(adjustment_param + ' is not a valid adjustment_param value')
    return 0

In [669]:
def get_adjustment_factor_param(adjustment_factor_param, distances):
    if adjustment_factor_param == 'average':
        return np.mean(distances)
    elif adjustment_factor_param == 'first_quartile':
        return np.percentile(distances, 25)
    elif adjustment_factor_param == 'median':
        return np.median(distances)
    elif adjustment_factor_param == 'third_quartile':
        return np.percentile(distances, 75)
    elif adjustment_factor_param == 'maximum':
        return np.max(distances)
    else:
        print(adjustment_factor_param + ' is not a valid cutoff_base value')
    return 0

In [670]:
# def get_background_lookup_table(df_distances, cutoff_base_param='maximum', adjustment_direction_param='add', 
#                                 adjustment_param='standard_deviation', adjustment_factor=2):
#     if make_new_lookup_table or new_grid_search:
#         # CHANGE THIS TO USE DEFAULT OR GIVEN PARAMETERS
#         # Create a new DataFrame with the same index and columns as df_distances
#         lookup_table = pd.DataFrame(index=df_distances.index, columns=df_distances.columns)
#         # Iterate through each cell in df_distances
#         for (height, azimuth), distances in df_distances.stack().items():
#             # If the list is not empty
#             if distances:
#                 # print(distances)
#                 # Get cutoff value
#                 # cutoff_base = get_cutoff_base(cutoff_base_param, distances)
#                 # adj_direction = get_adjustment_direction_param(adjustment_direction_param, distances)
#                 # adj = get_adjustment_parameter_param(adjustment_param, distances)
                
#                 # value = adj_direction * (adj * adjustment_factor) + cutoff_base
#                 value = np.max(distances) - 0.5
#                 # print(value)
#             else:
#                 # print('empty', end=', ')
#                 value = np.nan  # If the list is empty, set the cell to NaN
    
#             # Set the value in the new DataFrame
#             lookup_table.at[height, azimuth] = value

#         if make_new_lookup_table: 
#             # Save the DataFrames
#             print('\nSaving lookup table')
#             lookup_table.to_pickle("sample_lookup_table.pkl")
#     else:
#         # Load the DataFrames
#         print('\nLoading lookup table')
#         lookup_table = pd.read_pickle("1000_frame_min_05_lookup_table_01_25.pkl")

#     return lookup_table

In [671]:
def get_background_lookup_table(df_z, cutoff_base_param='maximum', adjustment_direction_param='add', 
                                adjustment_param='standard_deviation', adjustment_factor=2):
    if make_new_lookup_table or new_grid_search:
        # CHANGE THIS TO USE DEFAULT OR GIVEN PARAMETERS
        # Create a new DataFrame with the same index and columns as df_distances
        lookup_table = pd.DataFrame(index=df_z.index, columns=df_z.columns)
        # Iterate through each cell in df_distances
        for (height, azimuth), z in df_z.stack().items():
            # If the list is not empty
            if z:
                # print(distances)
                # Get cutoff value
                # cutoff_base = get_cutoff_base(cutoff_base_param, distances)
                # adj_direction = get_adjustment_direction_param(adjustment_direction_param, distances)
                # adj = get_adjustment_parameter_param(adjustment_param, distances)
                
                # value = adj_direction * (adj * adjustment_factor) + cutoff_base
                value = np.percentile(z, 75)
                # print(value)
            else:
                # print('empty', end=', ')
                value = np.nan  # If the list is empty, set the cell to NaN
    
            # Set the value in the new DataFrame
            lookup_table.at[height, azimuth] = value

        if make_new_lookup_table: 
            # Save the DataFrames
            print('\nSaving lookup table')
            lookup_table.to_pickle("height_lookup_table.pkl")
    else:
        # Load the DataFrames
        print('\nLoading lookup table')
        lookup_table = pd.read_pickle("height_lookup_table.pkl")

    return lookup_table

In [672]:
dir = Path(DATA_DIR_ROOT)

df_distances = get_distances_dataframe(dir)


Skip loading dataframe


In [673]:
if make_new_distances_df: 
    # Save the DataFrames
    print('\nSaving dataframe')
    df_distances.to_pickle("df_heights.pkl")

## Filtering and saving functions

In [675]:
def convert_to_dataframe(bin_path):
    pre_filtered_data = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4) 
    columns = ['x', 'y', 'z', 'intensity']
    df = pd.DataFrame(pre_filtered_data, columns=columns)
    return df

In [676]:
def add_lookup_coords_to_xyz(points_df):
    # print(points_df)
    # Calculate the distance, azimuth, and height using vectorized operations
    x, y, z, intensity = points_df['x'], points_df['y'], points_df['z'], points_df['intensity']
    distance = np.sqrt(x**2 + y**2 + z**2)
    azimuth = np.degrees(np.arctan2(y, x))
    # print(azimuth)
    height = np.degrees(np.arctan2(z, np.sqrt(x**2 + y**2)))
    
    # Convert and scale
    azimuth_idx = (np.floor((azimuth + 180) / azimuth_resolution) * azimuth_resolution * 10 - 1800).astype(int)
    height_idx = (np.floor((height + 30) / height_resolution) * height_resolution * 10 - 300).astype(int)
    
    # Add new columns to dataframe
    points_df['height'] = height
    points_df['azimuth_idx'] = azimuth_idx
    points_df['height_idx'] = height_idx

    # It would be nice to also add a delta z here
    # This would be roughly height angle / 55
    points_df['delta_z'] = height / 55
    
    return points_df

In [677]:
# def filter_points(frame_path, lookup_table):
#     pre_filtered_points = convert_to_dataframe(frame_path)
#     # Add lookup table coordinates
#     pre_filtered_grid_lookup = add_lookup_coords_to_xyz(pre_filtered_points)
    
#     # Convert 'azimuth_idx' and 'height_idx' to integers
#     pre_filtered_grid_lookup['azimuth_idx'] = pre_filtered_grid_lookup['azimuth_idx'].astype(int)
#     pre_filtered_grid_lookup['height_idx'] = pre_filtered_grid_lookup['height_idx'].astype(int)
    
#     # Set index to ['height_idx', 'azimuth_idx']
#     pre_filtered_grid_lookup_indexed = pre_filtered_grid_lookup.set_index(['height_idx', 'azimuth_idx'])
    
#     # Create a Series from the lookup table with a MultiIndex
#     lookup_series = lookup_table.stack()
    
#     # Reindex the lookup values to align with the DataFrame's index
#     lookup_values = lookup_series.reindex(pre_filtered_grid_lookup_indexed.index)
    
#     # Create a mask for indices that exist in the lookup_table
#     indices_in_lookup = lookup_values.index.isin(lookup_series.index)
    
#     # Create the condition based on your filtering criteria
#     condition = (
#         indices_in_lookup & (
#             lookup_values.isna() | 
#             (abs(pre_filtered_grid_lookup_indexed['x']) < abs(lookup_values) - pre_filtered_grid_lookup_indexed['delta_z']) 
#         )
#     )
    
#     # Apply the condition to filter the DataFrame
#     filtered_df = pre_filtered_grid_lookup_indexed[condition].reset_index()
    
#     return filtered_df


In [678]:
# import pandas as pd

# def filter_points(frame_path, lookup_table):
#     pre_filtered_points = convert_to_dataframe(frame_path)
#     pre_filtered_grid_lookup = add_lookup_coords_to_xyz(pre_filtered_points)
    
#     # Convert 'azimuth_idx' and 'height_idx' to strings if necessary (depends on how they are stored in lookup_table)
#     pre_filtered_grid_lookup['azimuth_idx'] = pre_filtered_grid_lookup['azimuth_idx'].astype(int)
#     pre_filtered_grid_lookup['height_idx'] = pre_filtered_grid_lookup['height_idx'].astype(int)

#     filtered = []

#     for index, point in pre_filtered_grid_lookup.iterrows():
#         azimuth_idx = int(point['azimuth_idx'])
#         height_idx = int(point['height_idx'])

#         # Ensure you are accessing the DataFrame by row and column labels
#         try:
#             bg_height = lookup_table.loc[height_idx, azimuth_idx]
#         except KeyError:
#             # print(f"KeyError for indices - Height: {height_idx}, Azimuth: {azimuth_idx}")
#             continue
        
#         # Calculation and comparison using absolute values and delta_z
#         point_effective_height = abs(point['z'] + point['delta_z'])
#         if point_effective_height < abs(bg_height):
#             filtered.append(point)
        
#         # Debugging output (optional)
#         # print(f"Point {index}: azimuth_idx={azimuth_idx}, height_idx={height_idx}, bg_height={bg_height}, point_height={point['height']}, delta_z={point['delta_z']}, effective_height={point_effective_height}")

#     # Return a DataFrame constructed from the filtered list
#     return pd.DataFrame(filtered)

In [679]:
def filter_points(frame_path, lookup_table):
    pre_filtered_points = convert_to_dataframe(frame_path)
    pre_filtered_grid_lookup = add_lookup_coords_to_xyz(pre_filtered_points)
    
    # Convert 'azimuth_idx' and 'height_idx' to integers
    pre_filtered_grid_lookup['azimuth_idx'] = pre_filtered_grid_lookup['azimuth_idx'].astype(int)
    pre_filtered_grid_lookup['height_idx'] = pre_filtered_grid_lookup['height_idx'].astype(int)

    # Set index to ['height_idx', 'azimuth_idx'] for alignment with lookup_table
    pre_filtered_grid_lookup.set_index(['height_idx', 'azimuth_idx'], inplace=True)
    
    # Flatten the lookup_table into a series with MultiIndex from its row and column indices
    lookup_series = lookup_table.stack()

    # Reindex the lookup values to align with the pre_filtered_grid_lookup index
    lookup_values = lookup_series.reindex(pre_filtered_grid_lookup.index)

    # Compute effective heights and absolute comparison within the DataFrame
    pre_filtered_grid_lookup['effective_height'] = abs(pre_filtered_grid_lookup['z'] + pre_filtered_grid_lookup['delta_z'])
    pre_filtered_grid_lookup['bg_height'] = lookup_values

    # Filter based on condition: check where effective height is less than the background height
    filtered_df = pre_filtered_grid_lookup[pre_filtered_grid_lookup['effective_height'] < abs(pre_filtered_grid_lookup['bg_height'])]

    # Reset index if you want 'height_idx' and 'azimuth_idx' as columns
    filtered_df = filtered_df.reset_index()

    return filtered_df

In [680]:
def save_as_binary(df, bin_path):
    try:
        # Ensure the DataFrame is in the correct order and data type
        data = df[['x', 'y', 'z', 'intensity']].astype(np.float32).values
        
        # Write the data to a binary file
        data.tofile(bin_path)
    except:
        print('could not save :' + str(bin_path))

In [681]:
def filter_frames(source_dir, save_dir, background_lookup_table, subset_size=1):   
    # Get just the file names
    files = [f for f in os.listdir(source_dir) if f.endswith('.bin')]

    if subset_size < 1:
        # Get subset of files to check
        random.shuffle(files)
        total_files = len(files)
        subset_num = int(total_files * subset_size)
    else:
        subset_num = len(files)
    print('Filtering frames ')
    # For each file
    for filename in files[:subset_num]:
        # Append file name to location
        print(filename, end=', ')
        from_file = Path(source_dir, filename)

        # Filter file
        filtered_df = filter_points(from_file, background_lookup_table)

        # APPEND FILE NAME TO NEW LOCATION
        to_file = Path(save_dir, filename)

        # CONVERT BACK TO BINARY and save
        save_as_binary(filtered_df, to_file)

## Grid search for lookup table

In [683]:
# Grid search parameters to find best lookup table
# parameters = {
#     'cutoff_base': ['average', 'first_quartile', 'median', 'third_quartile', 'maximum'],
#     'adjustment_direction': ['add', 'subtract'],
#     'adjustment_parameter': ['constant', 'standard_deviation'],
#     'adjustment_factor': [0, 0.5, 1, 1.5, 2]
# }

parameters = {
    'cutoff_base': ['average', 'median', 'third_quartile', 'maximum'],
    'adjustment_direction': ['add', 'subtract'],
    'adjustment_parameter': ['standard_deviation'],
    'adjustment_factor': [0, 0.5, 1, 1.5, 2]
}

parameters = {
    'cutoff_base': ['average', 'median', 'third_quartile', 'maximum'],
    'adjustment_direction': ['add', 'subtract'],
    'adjustment_parameter': ['standard_deviation'],
    'adjustment_factor': [0, 0.5, 1, 1.5, 2]
}

In [684]:
# Generate all combinations of parameters
all_combinations = list(itertools.product(
    parameters['cutoff_base'],
    parameters['adjustment_direction'],
    parameters['adjustment_parameter'],
    parameters['adjustment_factor']
))

In [685]:
for combination in all_combinations:
    print(combination)

('average', 'add', 'standard_deviation', 0)
('average', 'add', 'standard_deviation', 0.5)
('average', 'add', 'standard_deviation', 1)
('average', 'add', 'standard_deviation', 1.5)
('average', 'add', 'standard_deviation', 2)
('average', 'subtract', 'standard_deviation', 0)
('average', 'subtract', 'standard_deviation', 0.5)
('average', 'subtract', 'standard_deviation', 1)
('average', 'subtract', 'standard_deviation', 1.5)
('average', 'subtract', 'standard_deviation', 2)
('median', 'add', 'standard_deviation', 0)
('median', 'add', 'standard_deviation', 0.5)
('median', 'add', 'standard_deviation', 1)
('median', 'add', 'standard_deviation', 1.5)
('median', 'add', 'standard_deviation', 2)
('median', 'subtract', 'standard_deviation', 0)
('median', 'subtract', 'standard_deviation', 0.5)
('median', 'subtract', 'standard_deviation', 1)
('median', 'subtract', 'standard_deviation', 1.5)
('median', 'subtract', 'standard_deviation', 2)
('third_quartile', 'add', 'standard_deviation', 0)
('third_quart

In [686]:
def delete_contents(folder_path):
    # Check if the folder exists
    if not os.path.isdir(folder_path):
        raise ValueError(f"Folder does not exist: {folder_path}")

    # Iterate over all files in the folder and delete them
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Removes the file or link
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

In [687]:
def get_best_params(grid_search_results):
    high_score = 0
    best_key = ''
    for param_combo_key in grid_search_results:
        score = 2.5 * grid_search_results[param_combo_key]['results']['percent_retained_label_points'] - \
                                        grid_search_results[param_combo_key]['results']['percent_retained_non_label_points']
        if score > high_score:
            high_score = score
            best_key = param_combo_key
    return grid_search_results[best_key]['parameters'], best_key

In [688]:
# Create a new folder for the filtered frames in the directory
filter_save_dir = Path(DATA_DIR_ROOT, 'sample_filtered_points')
filter_save_dir.mkdir(exist_ok=True)

lidar_dir = Path(DATA_DIR_ROOT, 'velodyne_points')
temp_lidar_dir = Path('temp_lidar')
temp_label_dir = Path('temp_label')
label_dir = Path(DATA_DIR_ROOT, 'labels')
print(lidar_dir)

..\data\velodyne_points


In [689]:
grid_search_results = {}

def find_best_lookup_table():
    # For each combination of parameters
    for params in all_combinations:
        print('Checking parameters: ' + str(params))
        # Make new lookup table
        lookup_table = get_background_lookup_table(df_distances, 
                                                   cutoff_base_param=params[0], 
                                                   adjustment_direction_param=params[1], 
                                                   adjustment_param=params[2], 
                                                   adjustment_factor=params[3])
        # Empty temp file
        delete_contents('temp_lidar')
        # delete_contents('temp_labels')
        time.sleep(3)
        # Filter subset of data
        filter_frames(lidar_dir, temp_lidar_dir, lookup_table, subset_size=.01)
        # Evaluate filtered data
        print('\nEvaluating results ')
        results = evaluate_filter(temp_lidar_dir, label_dir, 'arcs_filtered')
        print()
        for key in results:
            print(key + ': ' + str(results[key]))
        print()

        # Create parameter combination key
        param_combo_key = params[0][0] + params[0][1] + '_' + params[1][0] + '_' + params[2][0] + '_' + str(params[3])
        # Store results in results dictionary
        grid_search_results[param_combo_key] = {'parameters': params,
                                                'results': results}

    best_params, best_params_key = get_best_params(grid_search_results)

    return best_params, best_params_key

In [690]:
grid_search_results = {}
# Load, create, or use grid search to find best lookup table
if new_grid_search:
    best_params, best_param_key = find_best_lookup_table()
    background_distance_lookup_table  = get_background_lookup_table(df_distances, 
                                                   cutoff_base_param=best_params[0], 
                                                   adjustment_direction_param=best_params[1], 
                                                   adjustment_param=best_params[2], 
                                                   adjustment_factor=best_params[3])
    print('\nSaving lookup table: ' + best_param_key)
    background_distance_lookup_table.to_pickle('lookup_table_' + best_param_key + '.pkl')
else:
    # Create background lookup table for distance cutoffs
    background_distance_lookup_table = get_background_lookup_table(df_distances)


Loading lookup table


In [691]:
print(grid_search_results)

{}


In [692]:
background_distance_lookup_table

,-1800,-1799,-1798,-1797,-1796,-1795,-1794,-1793,-1792,-1791,...,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799
-300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [693]:
# filled_lookup_table = background_distance_lookup_table.fillna(method='ffill', axis=0).fillna(method='bfill', axis=0)  # Fill along rows
# filled_lookup_table = filled_lookup_table.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1) 
filled_lookup_table = background_distance_lookup_table.ffill(axis=0).bfill(axis=0)  # Fill along rows
filled_lookup_table = filled_lookup_table.ffill(axis=1).bfill(axis=1)

C:\Users\gilya\AppData\Local\Temp\ipykernel_1640\2900473312.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filled_lookup_table = background_distance_lookup_table.ffill(axis=0).bfill(axis=0)  # Fill along rows


In [694]:
filled_lookup_table

,-1800,-1799,-1798,-1797,-1796,-1795,-1794,-1793,-1792,-1791,...,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799
-300,-1.363367,-0.564618,-1.617783,-0.575606,-0.557011,-1.616093,-1.617783,-0.546023,-0.600118,-0.844814,...,-1.621164,-0.549404,-1.354069,-0.549404,-1.363367,-0.555321,-1.617783,-0.561237,-1.617783,-1.614825
-297,-1.363367,-0.564618,-1.617783,-0.575606,-0.557011,-1.616093,-1.617783,-0.546023,-0.600118,-0.844814,...,-1.621164,-0.549404,-1.354069,-0.549404,-1.363367,-0.555321,-1.617783,-0.561237,-1.617783,-1.614825
-295,-1.363367,-0.564618,-1.617783,-0.575606,-0.557011,-1.616093,-1.617783,-0.546023,-0.600118,-0.844814,...,-1.621164,-0.549404,-1.354069,-0.549404,-1.363367,-0.555321,-1.617783,-0.561237,-1.617783,-1.614825
-292,-1.363367,-0.564618,-1.617783,-0.575606,-0.557011,-1.616093,-1.617783,-0.546023,-0.600118,-0.844814,...,-1.621164,-0.549404,-1.354069,-0.549404,-1.363367,-0.555321,-1.617783,-0.561237,-1.617783,-1.614825
-290,-1.363367,-0.564618,-1.617783,-0.575606,-0.557011,-1.616093,-1.617783,-0.546023,-0.600118,-0.844814,...,-1.621164,-0.549404,-1.354069,-0.549404,-1.363367,-0.555321,-1.617783,-0.561237,-1.617783,-1.614825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0.812625,0.811650,0.810675,0.809700,0.809213,0.808238,0.807263,0.806288,0.805313,0.804338,...,0.821887,0.820912,0.819450,0.818475,0.817500,0.816525,0.815550,0.814575,0.813600,0.812625
92,0.812625,0.811650,0.810675,0.809700,0.809213,0.808238,0.807263,0.806288,0.805313,0.804338,...,0.821887,0.820912,0.819450,0.818475,0.817500,0.816525,0.815550,0.814575,0.813600,0.812625
95,0.812625,0.811650,0.810675,0.809700,0.809213,0.808238,0.807263,0.806288,0.805313,0.804338,...,0.821887,0.820912,0.819450,0.818475,0.817500,0.816525,0.815550,0.814575,0.813600,0.812625
97,0.812625,0.811650,0.810675,0.809700,0.809213,0.808238,0.807263,0.806288,0.805313,0.804338,...,0.821887,0.820912,0.819450,0.818475,0.817500,0.816525,0.815550,0.814575,0.813600,0.812625


In [695]:
print(filled_lookup_table.index)
print(filled_lookup_table.columns)

Index([-300, -297, -295, -292, -290, -287, -285, -282, -280, -277,
       ...
         77,   80,   82,   85,   87,   90,   92,   95,   97,  100],
      dtype='int64', length=161)
Index([-1800, -1799, -1798, -1797, -1796, -1795, -1794, -1793, -1792, -1791,
       ...
        1790,  1791,  1792,  1793,  1794,  1795,  1796,  1797,  1798,  1799],
      dtype='int64', length=3600)


In [696]:
# Filter and save each filtered frame
# save_dir = Path(DATA_DIR_ROOT, 'test_folder')
save_dir = Path(DATA_DIR_ROOT, 'filtered_points')
# filter_frames(lidar_dir, save_dir, filled_lookup_table, subset_size=0.001)
filter_frames(lidar_dir, save_dir, filled_lookup_table)

Filtering frames 
006453.bin, 006447.bin, 006450.bin, 006444.bin, 006454.bin, 006445.bin, 006448.bin, 006456.bin, 006446.bin, 006449.bin, 006452.bin, 006451.bin, 006455.bin, 006442.bin, 006443.bin, 006441.bin, 006439.bin, 006438.bin, 006437.bin, 006440.bin, 006436.bin, 006386.bin, 006387.bin, 006385.bin, 006383.bin, 006381.bin, 006378.bin, 006379.bin, 006382.bin, 006380.bin, 006375.bin, 006384.bin, 006373.bin, 006376.bin, 006374.bin, 006370.bin, 006377.bin, 006371.bin, 006372.bin, 006368.bin, 006367.bin, 006364.bin, 006366.bin, 006362.bin, 006363.bin, 006361.bin, 006369.bin, 006365.bin, 006357.bin, 006359.bin, 006358.bin, 006355.bin, 006354.bin, 006351.bin, 006353.bin, 006356.bin, 006360.bin, 006352.bin, 006350.bin, 006348.bin, 006347.bin, 006343.bin, 006344.bin, 006346.bin, 006345.bin, 006349.bin, 006338.bin, 006339.bin, 006342.bin, 006340.bin, 006341.bin, 006335.bin, 006336.bin, 006334.bin, 006337.bin, 006330.bin, 006333.bin, 006331.bin, 006332.bin, 006328.bin, 006327.bin, 006329.bin